In [7]:
import cv2
import os

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Function to preprocess an image
def preprocess_image(image_path, output_path, target_size=(150, 150)):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        return

    # Convert to grayscale (optional but recommended)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        print(f"No face detected in {image_path}")
        return

    # Assume the first face is the target
    (x, y, w, h) = faces[0]

    # Crop the face
    face = gray[y:y+h, x:x+w]

    # Resize the face to the target size
    resized = cv2.resize(face, target_size, interpolation=cv2.INTER_AREA)

    # Save the preprocessed image
    cv2.imwrite(output_path, resized)
    print(f"Processed and saved {output_path}")

# Function to preprocess the entire dataset
def preprocess_dataset(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for person_name in os.listdir(input_folder):
        person_folder = os.path.join(input_folder, person_name)
        output_person_folder = os.path.join(output_folder, person_name)
        if not os.path.exists(output_person_folder):
            os.makedirs(output_person_folder)

        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            output_path = os.path.join(output_person_folder, image_name)
            preprocess_image(image_path, output_path)

# Preprocess the dataset
input_folder = "dataset"  # Folder containing raw images
output_folder = "preprocessed_dataset"  # Folder to save preprocessed images
preprocess_dataset(input_folder, output_folder)

Error: Unable to load image dataset\.git\.COMMIT_EDITMSG.swp
Error: Unable to load image dataset\.git\COMMIT_EDITMSG
Error: Unable to load image dataset\.git\config
Error: Unable to load image dataset\.git\description
Error: Unable to load image dataset\.git\HEAD
Error: Unable to load image dataset\.git\hooks
Error: Unable to load image dataset\.git\index
Error: Unable to load image dataset\.git\info
Error: Unable to load image dataset\.git\logs
Error: Unable to load image dataset\.git\objects
Error: Unable to load image dataset\.git\refs
Processed and saved preprocessed_dataset\Aniket-22010103005\Aniket-22010103005_0.jpg
No face detected in dataset\Aniket-22010103005\Aniket-22010103005_1.jpg
No face detected in dataset\Aniket-22010103005\Aniket-22010103005_2.jpg
Processed and saved preprocessed_dataset\Aryan Mahajan - 22010103011\Aryan Mahajan - 22010103011_0.jpg
No face detected in dataset\Aryan Mahajan - 22010103011\Aryan Mahajan - 22010103011_1.jpg
No face detected in dataset\Aryan

In [ ]:
import cv2
import os
import numpy as np
import dlib
from imutils import face_utils

# Initialize face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Download this file

# Function to preprocess an image
def preprocess_image(image_path, output_path, target_size=(160, 160)):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)
    if len(faces) == 0:
        print(f"No face detected in {image_path}")
        return

    # Assume the first face is the target
    face = faces[0]

    # Detect facial landmarks
    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)

    # Extract the eyes and nose for alignment
    left_eye = shape[36:42]
    right_eye = shape[42:48]
    nose = shape[27:31]

    # Calculate the center of the eyes
    left_eye_center = left_eye.mean(axis=0).astype(int)
    right_eye_center = right_eye.mean(axis=0).astype(int)

    # Calculate the angle between the eyes
    dY = right_eye_center[1] - left_eye_center[1]
    dX = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Calculate the center of the face
    face_center = (face.left() + face.right()) // 2, (face.top() + face.bottom()) // 2

    # Rotate the image to align the eyes
    M = cv2.getRotationMatrix2D(face_center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)

    # Crop the face
    (x, y, w, h) = cv2.boundingRect(np.array([left_eye_center, right_eye_center, nose.mean(axis=0)]))
    cropped = rotated[y:y+h, x:x+w]

    # Resize the face to the target size
    resized = cv2.resize(cropped, target_size, interpolation=cv2.INTER_AREA)

    # Normalize lighting
    resized = cv2.equalizeHist(cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY))

    # Save the preprocessed image
    cv2.imwrite(output_path, resized)
    print(f"Processed and saved {output_path}")

# Function to preprocess the entire dataset
def preprocess_dataset(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for person_name in os.listdir(input_folder):
        person_folder = os.path.join(input_folder, person_name)
        output_person_folder = os.path.join(output_folder, person_name)
        if not os.path.exists(output_person_folder):
            os.makedirs(output_person_folder)

        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            output_path = os.path.join(output_person_folder, image_name)
            preprocess_image(image_path, output_path)

# Preprocess the dataset
input_folder = "dataset"  # Folder containing raw images
output_folder = "preprocessed_dataset"  # Folder to save preprocessed images
preprocess_dataset(input_folder, output_folder)